In [47]:
import numpy as np
import math
import random as rn
import pandas as pd
from library import Map, GradientDescent
import time
from random import randrange
import scipy.spatial.distance as d
import heapq as hq

In [78]:
'''Downloading dataset'''
dataset = pd.read_csv("/Users/alessandrococcia/Downloads/ObesityDataSet.csv")

'''Dataset sampling'''
dataset = dataset.sample(frac=1, ignore_index=True) #shuffle sample in the training set

'''mapping delle stringhe'''
dataset.replace(to_replace=("Insufficient_Weight", "Normal_Weight", "Overweight_Level_I", "Overweight_Level_II"), value=0, inplace=True)
dataset.replace(to_replace=("Obesity_Type_I", "Obesity_Type_II", "Obesity_Type_III"), value=1, inplace=True)
dataset = pd.get_dummies(dataset, drop_first=True, dtype=float)

'''normalization'''
dataset = (dataset-dataset.mean())/dataset.std()

'''Inserimento colonna di bias'''
dataset.insert(0, "Bias", np.ones(len(dataset)), True) #Bias row


TRAIN_TEST_SPLIT_PERCENTAGE = 0.9
dataset_training = dataset[:int(len(dataset) * TRAIN_TEST_SPLIT_PERCENTAGE)]
dataset_test = dataset[int(len(dataset) * TRAIN_TEST_SPLIT_PERCENTAGE):]

/var/folders/yq/gyf11wn125qgkzlcv24mbp3w0000gn/T/ipykernel_88117/3992257774.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset.replace(to_replace=("Obesity_Type_I", "Obesity_Type_II", "Obesity_Type_III"), value=1, inplace=True)


In [81]:
features_len = len(dataset_test.columns)
len_test = len(dataset_test)
column_output_index = dataset_test.columns.get_loc("NObeyesdad")

count = 0
k = 1

X = dataset_training.loc[:, dataset_training.columns != 'NObeyesdad']
Y = dataset_training.loc[:, dataset_training.columns == 'NObeyesdad']
Z = dataset_test.loc[:, dataset_test.columns != 'NObeyesdad']


for j in range(1899, len_test+1899):
    minimum = math.inf
    ret = None
    sample = dataset_test.loc[j:j]
    '''initializing minheap for knn'''
    minheap = []
    hq.heapify(minheap)
    for i in range(0, len(dataset_training)):
        x = dataset.loc[i:i]
        distance = d.minkowski(np.array(sample)[0], np.array(x)[0], features_len)
        hq.heappush(minheap, (distance, list(x["NObeyesdad"])))
        if distance < minimum:
            minimum = distance
            ret = x["NObeyesdad"]
    
    '''ksmallest'''
    ksmallest = hq.nsmallest(k, minheap)
    max_value = max(list(zip(*ksmallest))[1])
    if max_value == np.array(ret):
        count += 1

print(count/len(dataset_test))


'''sklearn'''
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=k)
neigh = neigh.fit(X,Y)
x_test = dataset_training.loc[:, dataset_training.columns != 'NObeyesdad']
sk_prediction  = neigh.predict(x_test)
print(sk_prediction)



ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.